In [5]:
import pandas as pd
import seaborn as sns

In [6]:
data1 = pd.read_csv('../climate_data/merged_weekly_avg_temp.csv')
print(data1.head())



     datetime    tempmax    tempmin       temp        dew   humidity  \
0  2000-01-02  47.600000  29.300000  41.450000  31.500000  68.350000   
1  2000-01-09  35.714286  21.342857  29.128571  19.485714  68.742857   
2  2000-01-16  38.871429  24.857143  32.785714  23.485714  69.671429   
3  2000-01-23  35.242857  18.414286  26.142857  15.528571  65.642857   
4  2000-01-30  32.000000  14.128571  23.357143  10.900000  60.342857   

     precip  windspeed  sealevelpressure  moonphase  
0  0.117000  10.050000       1020.850000   0.830000  
1  0.109143   9.971429       1022.157143   0.550000  
2  0.037714  10.000000       1022.942857   0.220000  
3  0.019571   9.742857       1027.285714   0.461429  
4  0.003857  10.800000       1032.157143   0.697143  


In [3]:
# Seaborn을 활용하여 각 속성간의 상관관계 분석하기

sns.pairplot(data = data1, vars=['tempmax','tempmin','temp','dew','humidity','precip','windspeed','sealevelpressure','moonphase'], kind = 'scatter', dropna=True)

  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [105]:
####################################################################
# [주의!!!!] 
# class 속성에 라벨을 추가하는 코드입니다.
# 실행 X
data1['class'] = 0
for index, row in data1.iterrows():
    # 섭씨 30도가 넘어가는 경우 : 폭염으로 판단
    if row['tempmax'] > 73:
        data1.at[index, 'class'] = 1
    # 섭씨 영하 10도보다 작은 경우 : 한파로 판단
    if row['tempmin'] < 20:
        data1.at[index, 'class'] = 1

output_file_path = "../climate_data/labeled_climate.csv"
data1.to_csv(output_file_path, index=False)
print("New CSV file 'labeled_climate.csv' has been created with the 'class' column.")
####################################################################


New CSV file 'labeled_climate.csv' has been created with the 'class' column.


In [78]:
!pip install scikit-learn

In [106]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 데이터 불러오기
data2 = pd.read_csv('../climate_data/labeled_climate.csv')

# 데이터 분리 (훈련 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(data2[['moonphase']], data2['class'], test_size=0.2, shuffle=True)

print("The Dimension of the trainset is ", X_train.shape)
print("The Dimension of the testset is ", X_test.shape)

The Dimension of the trainset is  (1011, 1)
The Dimension of the testset is  (253, 1)


In [107]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [108]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(tol=1e-04).fit(X_train_scaled, y_train)

In [109]:
print("--------------------------------------------")
print("regression coefficients: ", model1.coef_)
print("intercept is ", model1.intercept_)
print("model score is ", model1.score(X_train_scaled, y_train))
print("--------------------------------------------")

--------------------------------------------
regression coefficients:  [[0.07689419]]
intercept is  [-0.00198102]
model score is  0.5153313550939663
--------------------------------------------


In [110]:
model1.predict_proba(X_test_scaled)

array([[0.47572654, 0.52427346],
       [0.48512524, 0.51487476],
       [0.50134659, 0.49865341],
       [0.50642452, 0.49357548],
       [0.50072729, 0.49927271],
       [0.4991171 , 0.5008829 ],
       [0.47387335, 0.52612665],
       [0.49589681, 0.50410319],
       [0.50023184, 0.49976816],
       [0.53016942, 0.46983058],
       [0.49007658, 0.50992342],
       [0.49986026, 0.50013974],
       [0.5315267 , 0.4684733 ],
       [0.47659159, 0.52340841],
       [0.53700023, 0.46299977],
       [0.47078633, 0.52921367],
       [0.53300684, 0.46699316],
       [0.50555762, 0.49444238],
       [0.4940391 , 0.5059609 ],
       [0.49874552, 0.50125448],
       [0.52399464, 0.47600536],
       [0.50939642, 0.49060358],
       [0.52621851, 0.47378149],
       [0.49354373, 0.50645627],
       [0.51509054, 0.48490946],
       [0.49663994, 0.50336006],
       [0.4725148 , 0.5274852 ],
       [0.50580531, 0.49419469],
       [0.50060343, 0.49939657],
       [0.49465832, 0.50534168],
       [0.

In [111]:
y_pred = model1.predict(X_test_scaled)
print("predicted class are ", y_pred)

predicted class are  [1 1 0 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 1
 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1
 1 1 1 0 0 1 0 1 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0 1
 1 0 1 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 1 0
 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0
 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1
 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 0 1 1 0 0 1 0]


In [112]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[55 66]
 [65 67]]


In [41]:
print(metrics.classification_report(y_test, y_pred, target_names=['class1', 'class2']))

              precision    recall  f1-score   support

      class1       0.89      1.00      0.94       225
      class2       0.00      0.00      0.00        28

    accuracy                           0.89       253
   macro avg       0.44      0.50      0.47       253
weighted avg       0.79      0.89      0.84       253



/Users/minseokkang/projects/statistic_project/statistic_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/minseokkang/projects/statistic_project/statistic_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/minseokkang/projects/statistic_project/statistic_venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete